In [57]:
import rankBN
import numpy as np
import qml
import math
import igraph as ig
import line_profiler
import numba
from numba.typed import List
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [2]:
%lprun -f rankBN.Ranker._get_site_similarity_CM rankBN.do_main("napthalene-molpro.xyz", "napthalene-molpro.mol2", 2.2, "CM") 

/home/guido/miniconda3/envs/analysis/lib/python3.6/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: C.ar
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


Working on stoichiometry: [5 5 5 5 5 7 7 7 7 7]
Found 66 molecules from 252 permutations.
55 components found
Group energy -2005.9177820267687
Found: [5, 7, 5, 7, 7, 5, 7, 5, 7, 5]
Group energy -1884.187380497132
Found: [5, 7, 5, 7, 5, 7, 7, 5, 7, 5]
Group energy -1884.187380497132
Found: [5, 7, 5, 7, 5, 7, 5, 7, 5, 7]
Group energy -1884.187380497132
Found: [5, 7, 5, 7, 7, 5, 5, 7, 7, 5]
Group energy -1884.1873804971317
Found: [5, 5, 7, 7, 7, 5, 7, 5, 7, 5]
Group energy -1838.4990439770554
Found: [5, 7, 5, 7, 7, 5, 7, 7, 5, 5]
Found: [5, 7, 5, 7, 7, 7, 5, 7, 5, 5]
Group energy -1838.4990439770554
Found: [7, 5, 5, 7, 7, 5, 7, 7, 5, 5]
Group energy -1808.145315487572
Found: [5, 5, 7, 5, 5, 7, 7, 5, 7, 7]
Group energy -1808.145315487572
Found: [5, 5, 7, 5, 5, 7, 5, 7, 7, 7]
Found: [5, 5, 7, 5, 7, 5, 7, 5, 7, 7]
Group energy -1762.4569789674956
Found: [5, 5, 7, 5, 5, 7, 7, 7, 5, 7]
Group energy -1762.4569789674956
Found: [5, 5, 7, 7, 5, 7, 5, 7, 5, 7]
Group energy -1762.4569789674956
Found

Timer unit: 1e-06 s

Total time: 8.58207 s
File: /mnt/c/Users/guido/workcopies/apdft/prototyping/networks/rankBN.py
Function: _get_site_similarity_CM at line 254

Line #      Hits         Time  Per Hit   % Time  Line Contents
   254                                           	def _get_site_similarity_CM(self, nuclear_charges):
   255                                           		""" Returns i, j, distance."""
   256      4169      37070.0      8.9      0.4  		charges = self._sitesimCM_charges
   257      4169      39727.0      9.5      0.5  		charges[self._includeonly] = nuclear_charges
   258      4169    2168908.0    520.2     25.3  		a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
   259                                           
   260                                           		# to squareform
   261      4169      87487.0     21.0      1.0  		self._sitesimCM_squareform_outcache[self._sitesimCM_squareform_mask] = 

In [18]:
nuclear_charges, coordinates = rankBN.Ranker.read_xyz("napthalene-molpro.xyz")
r = rankBN.Ranker(nuclear_charges, coordinates, "napthalene-molpro.xyz", "napthalene-molpro.mol2", explain=True, sim=2.2, simmode="CM")
r.rank()

Working on stoichiometry: [5 5 5 5 5 7 7 7 7 7]
Found 66 molecules from 252 permutations.
55 components found
Group energy -2005.9177820267687
Found: [5, 7, 5, 7, 7, 5, 7, 5, 7, 5]
Group energy -1884.187380497132
Found: [5, 7, 5, 7, 5, 7, 7, 5, 7, 5]
Group energy -1884.187380497132
Found: [5, 7, 5, 7, 5, 7, 5, 7, 5, 7]
Group energy -1884.187380497132
Found: [5, 7, 5, 7, 7, 5, 5, 7, 7, 5]
Group energy -1884.1873804971317
Found: [5, 5, 7, 7, 7, 5, 7, 5, 7, 5]
Group energy -1838.4990439770554
Found: [5, 7, 5, 7, 7, 5, 7, 7, 5, 5]
Found: [5, 7, 5, 7, 7, 7, 5, 7, 5, 5]
Group energy -1838.4990439770554
Found: [7, 5, 5, 7, 7, 5, 7, 7, 5, 5]
Group energy -1808.145315487572
Found: [5, 5, 7, 5, 5, 7, 7, 5, 7, 7]
Group energy -1808.145315487572
Found: [5, 5, 7, 5, 5, 7, 5, 7, 7, 7]
Found: [5, 5, 7, 5, 7, 5, 7, 5, 7, 7]
Group energy -1762.4569789674956
Found: [5, 5, 7, 5, 5, 7, 7, 7, 5, 7]
Group energy -1762.4569789674956
Found: [5, 5, 7, 7, 5, 7, 5, 7, 5, 7]
Group energy -1762.4569789674956
Found

In [31]:
# branch prediction
def test(self):
    nmolecules = len(self._molecules)
    graph = ig.Graph(nmolecules)
    for mol_i in range(nmolecules):
        for mol_j in range(mol_i, nmolecules):
            # short-circuit if other relations already exist
            if not math.isinf(graph.shortest_paths(mol_i, mol_j)[0][0]):
                continue

            for origin in self._molecules[mol_i]:
                for opposite in self._molecules[mol_j]:
                    # skip odd numbers of mutated sites
                    if (len(np.where(np.array(origin) != np.array(opposite))[0]) % 2) == 1:
                        continue

                    # check necessary requirements
                    reference = (np.array(opposite) + origin) / 2
                    check = precheck(self, origin, opposite, reference)
                    if check == False:
                        continue

                    common_ground = self._identify_equivalent_sites(reference)
                    if self._check_common_ground(*check, common_ground):
                        graph.add_edge(mol_i, mol_j)
                        break
                else:
                    continue
                break
                
def precheck(self, target, opposite, reference):
    deltaZ = [a-b for a,b in zip(opposite, target)]

    # at least of symmetry of all-carbon center
    deltaZ = np.array(deltaZ)
    for group in self._cache_precheck_similar:
        if sum(deltaZ[group]) != 0:
            return False
        
    # matching deltaZ
    changes = np.bincount([_ + 2 for _ in deltaZ], minlength=5)

    # ensure matching counts
    if max(changes - changes[::-1]) != 0:
        return False

    # ignore identity operation
    if changes[2] == self._nmodifiedatoms:
        return False

    return deltaZ, changes
r._precheck = precheck
%lprun -f precheck test(r)

Timer unit: 1e-06 s

Total time: 3.82384 s
File: <ipython-input-31-b04e1955197d>
Function: precheck at line 30

Line #      Hits         Time  Per Hit   % Time  Line Contents
    30                                           def precheck(self, target, opposite, reference):
    31     31115     648771.0     20.9     17.0      deltaZ = [a-b for a,b in zip(opposite, target)]
    32                                           
    33                                               # at least of symmetry of all-carbon center
    34     31115     500863.0     16.1     13.1      deltaZ = np.array(deltaZ)
    35     49648     376247.0      7.6      9.8      for group in self._cache_precheck_similar:
    36     45481    1679514.0     36.9     43.9          if deltaZ[group].sum() != 0:
    37     26948     196658.0      7.3      5.1              return False
    38                                                   
    39                                               # matching deltaZ
    40      416

In [80]:
@numba.jit(nopython=True)
def precheck(cache, target, opposite, reference, nmod):
    # at least of symmetry of all-carbon center
    s = 0
    for group in cache:
        for i in group:
            s += opposite[i] - target[i]
        if s != 0:
            return False
        
    deltaZ = np.array(opposite) - np.array(target)
    
    # matching deltaZ
    changes = np.zeros(5)
    for i in deltaZ:
        changes[i +2] +=1 

    # ensure matching counts
    if max(changes - changes[::-1]) != 0:
        return False

    # ignore identity operation
    if changes[2] == nmod:
        return False

    return True

a = tuple([5, 5, 5, 5, 5, 7, 7, 7, 7, 7])
%timeit precheck(cache, a, a,a, r._nmodifiedatoms)

33.5 µs ± 13.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
cache = List()
for group in r._cache_precheck_similar:
    group_list = List()
    for element in group:
        group_list.append(element)
    cache.append(group_list)

In [82]:
%timeit precheck(cache, a, a,a, r._nmodifiedatoms)

8.53 µs ± 211 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [90]:
1000*1000/3

333333.3333333333

In [91]:
500000000/333333

1500.0015000015

In [72]:
cache

ListType[ListType[int64]]([[0, 3, 4, 7], [1, 2, 5, 6], [8, 9]])

In [94]:
@numba.jit(nopython=True)
def precheck(target, opposite):
    if opposite[0] - target[0] + opposite[3] - target[3] + opposite[4] - target[4] + opposite[7] - target[7] != 0:
        return False
    if opposite[1] - target[1] + opposite[2] - target[2] + opposite[5] - target[5] + opposite[6] - target[6] != 0:
        return False
    if opposite[8] - target[8] + opposite[9] - target[9] != 0:
        return False
        
    deltaZ = np.array(opposite) - np.array(target)
    
    # matching deltaZ
    changes = np.zeros(5, dtype=np.int32)
    for i in deltaZ:
        changes[i +2] +=1 

    # ensure matching counts
    if max(changes - changes[::-1]) != 0:
        return False

    # ignore identity operation
    if changes[2] == 10:
        return False

    return True

a = tuple([5, 5, 5, 5, 5, 7, 7, 7, 7, 7])
%timeit precheck(a, a)

14.1 µs ± 6.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [96]:
%timeit precheck(a, a)

4.93 µs ± 365 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
